In [1]:
import numpy as np
from scipy import stats
from statistics import mean 
import random

In [2]:
def getSigma(averageQuality):
    if(averageQuality < 40 or averageQuality > 110):
        return 2
    elif(averageQuality < 50 or averageQuality > 100):
        return 5
    elif(averageQuality < 70 or averageQuality > 90):
        return 10
    else:
        return 15

def generateQuality(averageQuality, readSize, readFile):
    sigma = getSigma(averageQuality)
    a, b = (33 - mu) / sigma, (126 - mu) / sigma
    for i in range(readSize):
        quality = (int(round(stats.truncnorm.rvs(a, b, scale=sigma, loc=mu))))
        readFile.write(quality)

In [ ]:
nucleotids = ['A', 'C', 'G', 'T']
complNucleotids = {
  "A": "T",
  "T": "A",
  "G": "C", 
  "C": "G"
}
    
def normalDistributionNumbersGenerator:

def numOfFragments(coverage, genomSize, readLength):
    return (coverage * genomSize) / (2 * readLength) #ovde nam je pisalo insertSize ali na netu nadjoh da je readLength, proveri formulu

def generateReads(refGenom, quality, coverage, readSize, insertSize):
    genomSize = len(refGenom)
    fragmentNumber = numOfFragments(coverage, genomSize, readLength)
    read1File = open("Read1.fastq","w")
    read2File = open("Read2.fastq", "w")
    for i in fragmentNumber:
        fragmentPosition = -1
        while (fragmentPosition < 0 and (fragmentPosition + insertsize) > genomSize):
            fragmentPosition = normalDistributionNumbersGenerator
        j = 0;
        while (j < readSize):
            refPos = fragmentPosition
            if (refGenom[refPos] == 'D'):
                refPos += 1
            elif (refGenom[refPos] & INS): #radimo inserciju
                read[j] = nucleotids[refGenom[refPos] & 3]
                j +=1
            elif (refGenom[refPos] & SNIP): #radimo zamenu
                read[j] = nucleotids[refGenom[refPos] & 3]
                j +=1
                refPos += 1
            else:
                read[j] = refGenom[refPos]
                j +=1
                refPos += 1
        
        read2File.write(readId) #dodati generisanje readId-a           
        read1File.write(read) 
        read1File.write("\n+\n")
        generateQuality(quality, readSize, read1File)
        
        j = 0
        while (j < readSize):
            refPos = fragmentPosition + insertSize
            if(refGenom[refPos] == D):
                refPos -= 1
            elif(refGenom[refPos] & INS): #radimo inserciju
                read[j] = complNucleotids[nucleotids[refGenom[refPos] & 3]]
                j += 1
            elif(refGenom[refPos] & SNIP): #radimo zamenu
                read[j] = complNucleotids[nucleotids[refGenom[refPos] & 3]]
                j +=1
                refPos -= 1
            else:
                read[j] = complNucleotids[refGenom[refPos]]
                j +=1
                refPos -= 1
                         
        read2File.write(readId)            
        read2File.write(read)
        read2File.write("\n+\n")               
        generateQuality(quality, readSize, read2File)
    
    read1File.close()
    read2File.close()

def insertMutations(refGenom, errorSnip, errorInDel):
    for nucleotids in refGenom:
        errorProb = random() # Random float:  0.0 <= x < 1.0
        if (errorProb <= (errorSnip + errorInDel)):
            if(errorProb <= errorSnip): #radi snip
                refGenom[i] = randrange(4) | 4 #randrange -> Integer from 0 to 3 inclusive
            else: #radi indel
                if (random() < 0.5): #radi inserciju
                    refGenom[i] = randrange(4)
                else: #radi deleciju
                    refGenom[i] = 'D'

def createPairEndPairs(refGenom, quality, coverage, readSize, insertSize, errorSnip, errorInDel):
    insertMutations(refGenom, errorSnip, errorInDel)
    generateReads(refGenom, quality, coverage, readSize, insertSize)



In [ ]:
if(7& 4 ):
    print('jes')
f = open('proba.fasta')
file_contents = f.read()
print(file_contents)
lista = list(file_contents)
lista[0] = 4
print(lista)
if(isinstance(lista[1], str)):
    if(lista[1] == 'D'): #delecija
    else #bez promene
elif(lista[1] & 4): #subs
else #insercija
    print('to')